Launch a Dask backend for parallel computing:

In [1]:
from dask.distributed import Client

Client()

Client Scheduler: tcp://127.0.0.1:45185 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 16.66 GB


Change CWD to avoid `..` in file paths:

In [2]:
import os

os.chdir("/home/data-scientist/elo-merchant-category-recommendation")

Import libraries required across the whole notebook:

In [3]:
import dask.dataframe as dd

Load the data:

In [4]:
kwargs = dict(parse_dates=["purchase_date"], usecols=["purchase_date", "month_lag"])

transactions = dict(old=dd.read_csv("data/0-preprocessed/old/chunk.*.csv", **kwargs),
                    new=dd.read_csv("data/0-preprocessed/new/chunk.*.csv", **kwargs))

Add synthetic feature `month_diff`:

In [5]:
import numpy as np
from datetime import datetime

for ddf in transactions.values():
    time_diff = datetime.today() - ddf.purchase_date
    month_diff = time_diff // np.timedelta64(1, "M")
    ddf["month_diff"] = ddf.month_lag + month_diff

Compute values of `month_diff` feature and save them to CSV:

In [6]:
feature_dir = "data/1-synthetic-features/month_diff"

for key, ddf in transactions.items():
    month_diff_df = ddf[["month_diff"]].compute()
    
    # for some reason dataframes can't be saved to feather with default index
    month_diff_df.reset_index(drop=True).to_feather(f"{feature_dir}/{key}.feather")